In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
p = r'C:\Users\223099055.HCAD\Downloads\MEKARI_REVIEWS.csv'
df = pd.read_csv(p)

In [ ]:
df.head()

In [ ]:
review_dir = Path("./reviews").resolve()
review_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
for i in range(10000):
    file_name = f"{df['review_id'].iloc[i]}.txt"
    file_path = review_dir / file_name
    r = f"Review:\n{df['review_text'].iloc[i]}"

    with file_path.open("w", encoding="utf-8") as f:
        f.write(str(r))

# Build bot

In [ ]:
from llama_index.core.evaluation import FaithfulnessEvaluator
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex

In [ ]:
filename_fn = lambda filename: {"file_name": filename}
documents = SimpleDirectoryReader("./reviews", file_metadata=filename_fn).load_data()

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [ ]:
from llama_index.embeddings.ollama import OllamaEmbedding

In [ ]:
ollama_embedding = OllamaEmbedding(
    model_name="tinyllama:latest",
    base_url="http://localhost:11434",
)

llm = Ollama(model="tinyllama:latest", request_timeout=120.0)

In [ ]:
Settings.embed_model = ollama_embedding

In [ ]:
Settings.llm = llm

In [ ]:
index = VectorStoreIndex.from_documents(documents)
evaluator = FaithfulnessEvaluator(llm=llm)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("In comparison to our application, which music streaming platform are users most likely to compare ours with?")
print(response)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
eval_result = evaluator.evaluate_response(response=response)

In [ ]:
eval_result